In [2]:
import pandas as pd
import numpy as np
import requests
import time
from opencage.geocoder import OpenCageGeocode
from math import radians, sin, cos, sqrt, atan2
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from scipy.spatial import distance_matrix

In [14]:
eb = pd.read_csv( r'C:\Users\User\Downloads\Data Azure\Ebonyi_with_clusters.csv')

In [15]:
eb

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File,address,latitude,longitude,cluster
0,EBONYI,ABAKALIKI,ABAKPA,11-01-01-002,ADAZI-ENU HALL II,150,750,True,-1,False,...,UNKNOWN,1,139,0,2,https://docs.inecelectionresults.net/elections...,"ADAZI-ENU HALL II, ABAKPA, ABAKALIKI, Ebonyi,...",6.32485,8.11368,0
1,EBONYI,ABAKALIKI,ABAKPA,11-01-01-003,CO-OPERATIVE SQUARE I,257,1045,True,-1,False,...,UNKNOWN,2,163,1,1,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,"CO-OPERATIVE SQUARE I, ABAKPA, ABAKALIKI, Ebon...",6.32485,8.11368,0
2,EBONYI,ABAKALIKI,ABAKPA,11-01-01-004,CO-OPERATIVE SQUARE II,203,750,True,-1,False,...,UNKNOWN,1,193,4,2,https://docs.inecelectionresults.net/elections...,"CO-OPERATIVE SQUARE II, ABAKPA, ABAKALIKI, Ebo...",6.32485,8.11368,0
3,EBONYI,ABAKALIKI,ABAKPA,11-01-01-006,NIBO/NSUKKA STR JUNCTION II,192,750,True,-1,False,...,UNKNOWN,0,0,0,0,https://docs.inecelectionresults.net/elections...,"NIBO/NSUKKA STR JUNCTION II, ABAKPA, ABAKALIKI...",6.32485,8.11368,0
4,EBONYI,ABAKALIKI,ABAKPA,11-01-01-007,VANCO HOTEL SPACE I,254,1200,True,-1,False,...,UNKNOWN,4,244,2,2,https://docs.inecelectionresults.net/elections...,"VANCO HOTEL SPACE I, ABAKPA, ABAKALIKI, Ebonyi...",6.32485,8.11368,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448,EBONYI,ISHIELU,EZILLO II,11-08-06-013,UMUAKPU P/GROUND II,119,361,True,-1,False,...,UNKNOWN,2,108,4,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,"UMUAKPU P/GROUND II, EZILLO II, ISHIELU, Ebony...",6.42714,7.83583,11
2449,EBONYI,ISHIELU,EZILLO II,11-08-06-014,UMUEZOKE PLAY GROUND II,74,214,True,-1,False,...,UNKNOWN,4,60,3,0,https://docs.inecelectionresults.net/elections...,"UMUEZOKE PLAY GROUND II, EZILLO II, ISHIELU, E...",6.42714,7.83583,11
2450,EBONYI,ISHIELU,AGBA,11-08-01-015,NGAMGBO ENYALI-IGWE TOWN HALL,17,26,True,-1,False,...,UNKNOWN,7,9,0,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,"NGAMGBO ENYALI-IGWE TOWN HALL, AGBA, ISHIELU, ...",6.42714,7.83583,11
2451,EBONYI,ISHIELU,"AZUINYABA ""A""",11-08-03-006,OHABUEZU PRI. SCHOOL,91,344,True,-1,False,...,UNKNOWN,13,63,6,2,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,"OHABUEZU PRI. SCHOOL, AZUINYABA ""A"", ISHIELU, ...",6.42714,7.83583,11


In [18]:
# Drop columns 'B' and 'D'
eb.drop(columns=['Results_File', 'cluster'])

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,address,latitude,longitude
0,EBONYI,ABAKALIKI,ABAKPA,11-01-01-002,ADAZI-ENU HALL II,150,750,True,-1,False,...,False,False,UNKNOWN,1,139,0,2,"ADAZI-ENU HALL II, ABAKPA, ABAKALIKI, Ebonyi,...",6.32485,8.11368
1,EBONYI,ABAKALIKI,ABAKPA,11-01-01-003,CO-OPERATIVE SQUARE I,257,1045,True,-1,False,...,False,False,UNKNOWN,2,163,1,1,"CO-OPERATIVE SQUARE I, ABAKPA, ABAKALIKI, Ebon...",6.32485,8.11368
2,EBONYI,ABAKALIKI,ABAKPA,11-01-01-004,CO-OPERATIVE SQUARE II,203,750,True,-1,False,...,False,False,UNKNOWN,1,193,4,2,"CO-OPERATIVE SQUARE II, ABAKPA, ABAKALIKI, Ebo...",6.32485,8.11368
3,EBONYI,ABAKALIKI,ABAKPA,11-01-01-006,NIBO/NSUKKA STR JUNCTION II,192,750,True,-1,False,...,True,False,UNKNOWN,0,0,0,0,"NIBO/NSUKKA STR JUNCTION II, ABAKPA, ABAKALIKI...",6.32485,8.11368
4,EBONYI,ABAKALIKI,ABAKPA,11-01-01-007,VANCO HOTEL SPACE I,254,1200,True,-1,False,...,False,False,UNKNOWN,4,244,2,2,"VANCO HOTEL SPACE I, ABAKPA, ABAKALIKI, Ebonyi...",6.32485,8.11368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448,EBONYI,ISHIELU,EZILLO II,11-08-06-013,UMUAKPU P/GROUND II,119,361,True,-1,False,...,False,False,UNKNOWN,2,108,4,0,"UMUAKPU P/GROUND II, EZILLO II, ISHIELU, Ebony...",6.42714,7.83583
2449,EBONYI,ISHIELU,EZILLO II,11-08-06-014,UMUEZOKE PLAY GROUND II,74,214,True,-1,False,...,False,False,UNKNOWN,4,60,3,0,"UMUEZOKE PLAY GROUND II, EZILLO II, ISHIELU, E...",6.42714,7.83583
2450,EBONYI,ISHIELU,AGBA,11-08-01-015,NGAMGBO ENYALI-IGWE TOWN HALL,17,26,True,-1,False,...,False,False,UNKNOWN,7,9,0,0,"NGAMGBO ENYALI-IGWE TOWN HALL, AGBA, ISHIELU, ...",6.42714,7.83583
2451,EBONYI,ISHIELU,"AZUINYABA ""A""",11-08-03-006,OHABUEZU PRI. SCHOOL,91,344,True,-1,False,...,False,False,UNKNOWN,13,63,6,2,"OHABUEZU PRI. SCHOOL, AZUINYABA ""A"", ISHIELU, ...",6.42714,7.83583


In [19]:
eb.shape

(2453, 23)

In [20]:
# subset the dataset
data = eb.loc[:, ['State','LGA','Ward','PU-Code','PU-Name','APC','LP','PDP','NNPP','latitude','longitude']].copy()
data

,State,LGA,Ward,PU-Code,PU-Name,APC,LP,PDP,NNPP,latitude,longitude
0,EBONYI,ABAKALIKI,ABAKPA,11-01-01-002,ADAZI-ENU HALL II,1,139,0,2,6.32485,8.11368
1,EBONYI,ABAKALIKI,ABAKPA,11-01-01-003,CO-OPERATIVE SQUARE I,2,163,1,1,6.32485,8.11368
2,EBONYI,ABAKALIKI,ABAKPA,11-01-01-004,CO-OPERATIVE SQUARE II,1,193,4,2,6.32485,8.11368
3,EBONYI,ABAKALIKI,ABAKPA,11-01-01-006,NIBO/NSUKKA STR JUNCTION II,0,0,0,0,6.32485,8.11368
4,EBONYI,ABAKALIKI,ABAKPA,11-01-01-007,VANCO HOTEL SPACE I,4,244,2,2,6.32485,8.11368
...,...,...,...,...,...,...,...,...,...,...,...
2448,EBONYI,ISHIELU,EZILLO II,11-08-06-013,UMUAKPU P/GROUND II,2,108,4,0,6.42714,7.83583
2449,EBONYI,ISHIELU,EZILLO II,11-08-06-014,UMUEZOKE PLAY GROUND II,4,60,3,0,6.42714,7.83583
2450,EBONYI,ISHIELU,AGBA,11-08-01-015,NGAMGBO ENYALI-IGWE TOWN HALL,7,9,0,0,6.42714,7.83583
2451,EBONYI,ISHIELU,"AZUINYABA ""A""",11-08-03-006,OHABUEZU PRI. SCHOOL,13,63,6,2,6.42714,7.83583


In [21]:
#check for the percentage of null values in each columns
(data.isnull().sum()/len(data)*100)
# The percentage of null values in both lat ang lon is 100 we need to drop those columns

State        0.0
LGA          0.0
Ward         0.0
PU-Code      0.0
PU-Name      0.0
APC          0.0
LP           0.0
PDP          0.0
NNPP         0.0
latitude     0.0
longitude    0.0
dtype: float64

In [23]:
data.dropna(subset=['latitude','longitude'],inplace = True)

In [30]:
# Extract latitude and longitude columns
lat_lon = data[['latitude', 'longitude']].values

# Calculate the distance matrix between polling units
dist_matrix = distance_matrix(lat_lon, lat_lon)

# Define a function to create a DataFrame with distances
def create_distance_dataframe(data, dist_matrix):
    # Create a new DataFrame to store distances
    distance_df = pd.DataFrame(dist_matrix, index=data.index, columns=data.index)
    
    # Convert the DataFrame to a more readable format
    distance_list = []
    for i in distance_df.index:
        for j in distance_df.columns:
            distance_list.append({
                'Polling_Unit_1': i,
                'Polling_Unit_2': j,
                'Distance_km': distance_df.at[i, j]
            })
    
    return pd.DataFrame(distance_list)

# Create the distance DataFrame
distance_df = create_distance_dataframe(data, dist_matrix)

# Merge the distance information back into the original DataFrame
data_with_distances = pd.merge(data, distance_df, left_index=True, right_on='Polling_Unit_1')


In [32]:
data_with_distances

,State,LGA,Ward,PU-Code,PU-Name,APC,LP,PDP,NNPP,latitude,longitude,Polling_Unit_1,Polling_Unit_2,Distance_km
0,EBONYI,ABAKALIKI,ABAKPA,11-01-01-002,ADAZI-ENU HALL II,1,139,0,2,6.32485,8.11368,0,0,0.0
1,EBONYI,ABAKALIKI,ABAKPA,11-01-01-002,ADAZI-ENU HALL II,1,139,0,2,6.32485,8.11368,0,1,0.0
2,EBONYI,ABAKALIKI,ABAKPA,11-01-01-002,ADAZI-ENU HALL II,1,139,0,2,6.32485,8.11368,0,2,0.0
3,EBONYI,ABAKALIKI,ABAKPA,11-01-01-002,ADAZI-ENU HALL II,1,139,0,2,6.32485,8.11368,0,3,0.0
4,EBONYI,ABAKALIKI,ABAKPA,11-01-01-002,ADAZI-ENU HALL II,1,139,0,2,6.32485,8.11368,0,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6017204,EBONYI,ISHIELU,EZZAGU I (OGBOJI),11-08-07-013,OHUHU PRI. SCHOOL,6,48,0,0,6.42714,7.83583,2452,2448,0.0
6017205,EBONYI,ISHIELU,EZZAGU I (OGBOJI),11-08-07-013,OHUHU PRI. SCHOOL,6,48,0,0,6.42714,7.83583,2452,2449,0.0
6017206,EBONYI,ISHIELU,EZZAGU I (OGBOJI),11-08-07-013,OHUHU PRI. SCHOOL,6,48,0,0,6.42714,7.83583,2452,2450,0.0
6017207,EBONYI,ISHIELU,EZZAGU I (OGBOJI),11-08-07-013,OHUHU PRI. SCHOOL,6,48,0,0,6.42714,7.83583,2452,2451,0.0
